In [1]:
import pandas as pd
from time import sleep
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_binary

from bs4 import BeautifulSoup

import requests
import pprint

In [2]:
def get_login():
    '''
    Access API keys from external file and placed in a list.

    Parameters
    ----------
    None:

    Returns
    ----------
    API_KEYS: (list)
        Return API keys used for connected to Spotify and Genius APIs.
    '''
    f = open('../data/LI_login.txt', 'r')
    creds = f.readlines()

    for idx, key in enumerate(creds):
        creds[idx] = key.replace('\n', '')

    return creds

In [3]:
def stock_scraper(stock):
    '''
    Scrapes passed in url (based on Yahoo Finance). Uses Selenium to
    complete loading stock history table.
    
    Parameters
    ----------
    stock : (str)
        A url to a webpage with a lazy loaded stock history table.

    Returns
    ----------
    mongo_dict : (dict)
        Dictionary ready for Mongo insert. Keys are dates, values are 
        lists of each days EOD trading results. 
    '''
    driver = webdriver.Chrome()
    driver.get(stock)
    sleep(10)
    for _ in range(30):
        sleep(1.5)          #random.randint for more randomized scrapping intervals
        driver.execute_script("window.scrollTo(0, window.scrollY + 3000)")
    r = driver.page_source
    soup = BeautifulSoup(r, 'html.parser')
    table_soup = soup.find_all('table')
    indices = table_soup[0].find_all('td')
    indices.pop(-1)
    mongo_dict = mongo_parser(indices)
    
    if mongo_dict: 
        driver.close()
        
    return mongo_dict

In [25]:
def li_login():
    # modeling flow below
    email, pw = get_login()
    driver = webdriver.Chrome()
    driver.get('https://www.linkedin.com/')
    driver.find_element_by_id('session_key').send_keys(email)
    driver.find_element_by_id('session_password').send_keys(pw)
    driver.find_element_by_id('session_password').send_keys(Keys.RETURN)
    return None

### defining flow for `li_login` function

In [3]:
email, pw = get_login()
# pull pw/login from external file
driver = webdriver.Chrome()
# inst chrome webdriver
driver.get('https://www.linkedin.com/')
# open LI page
driver.find_element_by_id('session_key').send_keys(email)
# pass in email
driver.find_element_by_id('session_password').send_keys(pw)
# pass in pw
'''TODO: .click() with xpath'''

# driver.find_element_by_id('homepage-basic_signin-form_submit-button').click()

'TODO: .click() with xpath'

In [5]:
driver.find_element_by_id('session_password').send_keys(Keys.RETURN)
# press enter for to log in

In [4]:
co_test = 'apple'

In [5]:
driver.find_element_by_xpath('//*[@id="ember16"]/input').send_keys(co_test)

In [6]:
driver.find_element_by_xpath('//*[@id="ember16"]/input').send_keys(Keys.RETURN)
# enter search for co. name

In [7]:
co_x_path = '/html/body/div[7]/div[3]/div/div[2]/div/div[2]/div/div/div/ul/li[1]/div/div/div[2]/a/h3'

In [8]:
driver.find_element_by_xpath(co_x_path).click()
# click co. link

In [9]:
ppl_x_path = '/html/body/div[7]/div[3]/div/div[3]/div[2]/div[1]/div/div/nav/ul/li[5]/a'

In [10]:
driver.find_element_by_xpath(ppl_x_path).click()
# click People

## ~~edit button testing~~
### moving to general search, b/c more flexibility in search

In [11]:
gen_search_xpath = '/html/body/div[7]/div[3]/div/div[3]/div[2]/div[2]/div[1]/div[2]/div/div/div/input'
gal_alum = 'galvanize'
tech_rec = 'technical recruiter'

In [12]:
driver.find_element_by_xpath(gen_search_xpath).send_keys(gal_alum)
driver.find_element_by_xpath(gen_search_xpath).send_keys(Keys.RETURN)

In [13]:
r = driver.page_source

In [14]:
soup = BeautifulSoup(r, 'html.parser')

In [94]:
results = soup.find('ul', 'org-people-profiles-module__profile-list')

In [95]:
len(results)

101

In [96]:
contact_elements = results.find_all('li', 'org-people-profiles-module__profile-item')

In [102]:
len(contact_elements)

25

### iterate through indecies of `contact_elements` to pull k-v pairs

In [97]:
contact_elements[0].find('div', 'org-people-profile-card__profile-title t-black lt-line-clamp lt-line-clamp--single-line ember-view')

<div class="org-people-profile-card__profile-title t-black lt-line-clamp lt-line-clamp--single-line ember-view" id="ember2874">  Erin Gong

<!-- --></div>

In [103]:
#k example
name = contact_elements[0].find('div', 'org-people-profile-card__profile-title t-black lt-line-clamp lt-line-clamp--single-line ember-view').text.rstrip()
name = name.replace(' ', '', 2)
name

'Erin Gong'

In [107]:
#v example
#link to contact
link = 'https://www.linkedin.com' + contact_elements[0].a['href']
link

'https://www.linkedin.com/in/eringong/'

In [ ]:
# for loop and dictionary pop init

Ok, interacting with LI just fine. ~~Need to refine 'add' edu "Galvanize" and~~ Move to general search for flexbility. Able to search co. for my desired fields... Need to scrape portion of return employees. 

--> beautifulsoup to scrape return cells for (k) employee name (v) url to profile

--> script Just portion. Just need DF out. for list of co's to search

--> MongoDB for k-v's

--> Dataframes

*perhaps load into psql for py wrapper struc. RDBMS experience*